In [144]:
import pandas as pd
import numpy as np
import re
output_path = r'C:\Users\ThinkPad\Desktop\河南\合并'
output_path = eval(repr(output_path))+str('\\')
# df = pd.read_csv(output_path + "合并.csv",encoding = 'utf-8',dtype = object)
df = pd.read_excel(output_path + "银行交易流水.xlsx",encoding = "gbk",dtype = object)
df = df.applymap(lambda x: re.sub('nan','',str(x)))
df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
df['交易余额'] = df['交易余额'].map(lambda x: float(x))

df['序号'] = df.index
all_流水完整 = df

# 客户公司五千万以上进款单笔追踪
# df['交易日期'] = df['交易时间'].str[0:10]


In [152]:
def 一方匹配另一方(df_一方, all_另一方):
    df_一方.index = df_一方['序号'].to_list()
    df_匹配 = pd.DataFrame()
    df_未匹配 = pd.DataFrame()
    df_另一方 = pd.DataFrame(all_另一方[ (all_另一方['借贷标志']=='出')  ])
# df_另一方 = pd.DataFrame(all_流水完整[ (all_流水完整['借贷标志']=='出') & ( all_流水完整['对手户名'].map(lambda  x:len(x)>3 or len(x)==0) )  ])
    print(str(len(df_一方))+"条，最大索引为"+ str(df_一方.index.max()))
# 差异 days = 1，minutes=10 ,seconds = 10

    for j in df_一方.index:
        for N in range(100):
            x = index向后N位的index(df_另一方, j, N)
    
            df_一进一出 = df_另一方[( df_另一方['序号'] <= x ) & ( df_另一方['序号'] > j ) \
                & (df_另一方['查询卡号']== df_一方['查询卡号'].loc[j] ) \
                ]
            if df_一方['交易金额'].loc[j]*(1-0.10) < df_一进一出['交易金额'].sum():
                break
            else:
                continue
        if len(df_一进一出)>0:
            merge_一进一出 = pd.merge(df_一方.loc[[j],:],df_一进一出,how = 'outer')
            df_匹配 = df_匹配.append(merge_一进一出)
            print(str(j)+"-"+str(x), end = ' ')
        elif len(df_一进一出)==0:
            df_未匹配 = df_未匹配.append(df_一方.loc[[j],:])
            print(str(j)+'失败', end = ' ')
    return df_匹配

In [147]:
def AB匹配(df_A,df_B):
    df_A.index = df_A['序号'].to_list()
    df_A出B进 = pd.DataFrame()
    df_A出B空 = pd.DataFrame()
    df_A出 = df_A[df_A['借贷标志']=='出'].drop_duplicates(subset = ['序号'],keep = 'last')
    for k in df_A出.index:
        df_B进 = df_B[(df_B['借贷标志']=='进') \
            & ( df_B['交易对方卡号'].isin(df_A出['查询卡号']) ) \
            & (df_B['交易时间']<= str(datetime.datetime.strptime(df_A出['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes = 1) )  ) \
            & (df_B['交易时间']>= str(datetime.datetime.strptime(df_A出['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(seconds = -10) ) ) \
            & (df_B['交易金额'] == df_A出['交易金额'].loc[k])  \
            ]
        if len(df_B进)>0:
            merge_A出B进 = pd.merge(df_A出.loc[[k],:], df_B进, how = 'outer')
            df_A出B进 = df_A出B进.append(merge_A出B进)
            print(k, end = ' ')
        elif len(df_B进)==0:
            df_A出B空 = df_A出B空.append(df_A出.loc[[k],:])
            print(str(k)+'失败', end = ' ')
    return df_A出B进

In [155]:
df_原点线索 = all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>50000000 )
                 & (all_流水完整['借贷标志']=='进')]

df_一层 = 一方匹配另一方(df_原点线索,all_流水完整)
df_一层.to_excel(output_path +  "多层匹配.xlsx",index = None)
print('Done!')

27条，最大索引为37744
36053-36054 36060-36062 36063-36067 36152-36154 36162-36181 36243-36244 36287-36292 36357-36358 36370-36371 36378-36380 36459-36461 36536-36580 36579-36580 36808-36815 36897-36898 36929-36930 37018-37020 37021-37023 37031-37032 37088-37091 37226-37227 37237-37239 37247-37251 37316-37318 37602-37603 37713-37714 37744-37745 Done!


In [156]:
df_一层A出B进 = AB匹配(df_一层,all_流水完整)
df_一层A出B进.to_excel(output_path + '一层A出B进.xlsx', sheet_name = '匹配',index = None)
print('Done!')

36054 36062失败 36064失败 36065失败 36066失败 36067失败 36153失败 36154失败 36163失败 36164 36165失败 36166 36167失败 36169失败 36170 36171失败 36172失败 36173 36174 36175 36178失败 36179失败 36180失败 36181失败 36244失败 36288 36292失败 36358 36371失败 36380失败 36460 36461 36537失败 36558失败 36559失败 36560失败 36561失败 36562失败 36563失败 36564失败 36565失败 36566失败 36567失败 36580失败 36812失败 36813失败 36814失败 36815失败 36898失败 36930失败 37020 37023 37032 37089 37090 37091 37227 37238 37239 37249失败 37250失败 37251 37317失败 37318 37603 37714失败 37745 Done!


In [157]:
df_一层线索 = df_一层A出B进[ (df_一层A出B进['借贷标志']=='进')]

df_二层 = 一方匹配另一方(df_一层线索,all_流水完整)

with pd.ExcelWriter(output_path +  "多层匹配.xlsx" ,mode='a', engine='openpyxl') as writer:
    df_二层.to_excel(writer, sheet_name= '二层',index = False)
print('Done!')

24条，最大索引为41513
3622-3624 3698-3699 3700-3701 24053-24055 3702-3703 37074-37076 37075-37076 3743-3746 37226-37227 37021-37023 37024-37026 36243-36244 36536-36580 36053-36054 36246-36247 3720-3721 36249-36250 41513-41521 36369-36371 36370-36371 36378-36380 26372-26385 36897-36898 36929-36930 Done!


In [158]:
df_二层A出B进 = AB匹配(df_二层,all_流水完整)
df_二层A出B进.to_excel(output_path + '二层A出B进.xlsx', sheet_name = '匹配',index = None)
print('Done!')

3623 3624失败 3699失败 3701失败 24054失败 24055失败 3703失败 37076失败 3744失败 3745失败 3746失败 37227 37023 37026 36244失败 36537失败 36558失败 36559失败 36560失败 36561失败 36562失败 36563失败 36564失败 36565失败 36566失败 36567失败 36580失败 36054 36247失败 3721 36250失败 41514失败 41515失败 41516失败 41517失败 41518失败 41520失败 41521失败 36371失败 36380失败 26384失败 26385 36898失败 36930失败 

In [159]:
df_二层线索 = df_二层A出B进[ (df_二层A出B进['借贷标志']=='进')]

df_三层 = 一方匹配另一方(df_二层线索,all_流水完整)

with pd.ExcelWriter(output_path +  "多层匹配.xlsx" ,mode='a', engine='openpyxl') as writer:
    df_三层.to_excel(writer, sheet_name= '三层',index = False)
print('Done!')

7条，最大索引为41513
68-71 41513-41521 36536-36580 36514-36516 3622-3624 36248-36250 25367-25369 Done!


In [160]:
df_三层A出B进 = AB匹配(df_三层, all_流水完整)
df_三层A出B进.to_excel(output_path + '三层A出B进.xlsx', sheet_name = '匹配',index = None)
print('Done!')

71失败 41514失败 41515失败 41516失败 41517失败 41518失败 41520失败 41521失败 36537失败 36558失败 36559失败 36560失败 36561失败 36562失败 36563失败 36564失败 36565失败 36566失败 36567失败 36580失败 36516 3623 3624失败 36250失败 25369失败 Done!


In [161]:
df_三层线索 = df_三层A出B进[ (df_三层A出B进['借贷标志']=='进')]

df_四层 = 一方匹配另一方(df_三层线索,all_流水完整)

with pd.ExcelWriter(output_path +  "多层匹配.xlsx" ,mode='a', engine='openpyxl') as writer:
    df_四层.to_excel(writer, sheet_name= '四层',index = False)
print('Done!')

2条，最大索引为24204
24204-24205 68-71 Done!


In [162]:
df_四层A出B进 = AB匹配(df_四层, all_流水完整)
df_四层A出B进.to_excel(output_path + '四层A出B进.xlsx', sheet_name = '匹配',index = None)
print('Done!')

24205失败 71失败 Done!


In [66]:
for j in [36053]:
    x = index向后N位的index(df_另一方, j, 0)
    
    df_一进一出 = df_另一方[( df_另一方['序号'] <= x ) & ( df_另一方['序号'] > j ) &\
        (df_另一方['查询卡号']== df_一方['查询卡号'].loc[j] ) \
#         & (df_另一方['交易时间']<= str(datetime.datetime.strptime(df_一方['交易时间'].loc[j], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(days = 1) )[0:10]+' 23:59:59'  )& \
#         (df_另一方['交易金额']>= df_一方['交易金额'].loc[j]*(1-0.005)) & \
#         (df_另一方['交易金额']<= df_一方['交易金额'].loc[j]*(1+0.005)) 
        ]
            
    if len(df_一进一出)>0:
        merge_一进一出 = pd.merge(df_一进一出,df_一方.loc[[j],:],how = 'outer')
        df_匹配 = df_匹配.append(merge_一进一出)
        print(str(j)+"-"+str(x), end = ' ')
    elif len(df_一进一出)==0:
        df_未匹配 = df_未匹配.append(df_一方.loc[[j],:])
        print(str(j)+'失败', end = ' ')



36053-36054 

In [78]:
for j in [36060]:
    for N in range(10):
        x = index向后N位的index(df_另一方, j, N)
    
        df_一进一出 = df_另一方[( df_另一方['序号'] <= x ) & ( df_另一方['序号'] > j ) \
            & (df_另一方['查询卡号']== df_一方['查询卡号'].loc[j] ) \
#         & (df_另一方['交易时间']<= str(datetime.datetime.strptime(df_一方['交易时间'].loc[j], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(days = 1) )[0:10]+' 23:59:59'  ) \
#         & (df_另一方['交易时间']>= str(datetime.datetime.strptime(df_一方['交易时间'].loc[j], '%Y-%m-%d %H:%M:%S') ) ) \
#         & (df_另一方['交易金额']>= df_一方['交易金额'].loc[j]*(1-0.005))  \
#         & (df_另一方['交易金额']<= df_一方['交易金额'].loc[j]*(1+0.005)) 
            ]
        if df_一方['交易金额'].loc[j] > df_一进一出['交易金额'].sum():
            print(str(df_一方['交易金额'].loc[j]) +"小于"+str(df_一进一出['交易金额'].sum())   )
            continue
        else:
            break
    if len(df_一进一出)>0:
        merge_一进一出 = pd.merge(df_一方.loc[[j],:],df_一进一出,how = 'outer')
        df_匹配 = df_匹配.append(merge_一进一出)
        print(str(j)+"-"+str(x), end = ' ')
    elif len(df_一进一出)==0:
        df_未匹配 = df_未匹配.append(df_一方.loc[[j],:])
        print(str(j)+'失败', end = ' ')

36060-36062 

In [181]:
df_A出 = all_流水完整.loc[[36064]]

for k in df_A出.index:
    df_B进 = all_流水完整[(all_流水完整['借贷标志']=='进') \
            & ( all_流水完整['交易对方卡号'].isin(df_A出['查询卡号']) ) \
            & (all_流水完整['交易时间']<= str(datetime.datetime.strptime(df_A出['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(seconds = 1) )  ) \
            & (all_流水完整['交易时间']>= str(datetime.datetime.strptime(df_A出['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(seconds = -1) ) ) \
            & (all_流水完整['交易金额'] == df_A出['交易金额'].loc[k])  \
            ]
    if len(df_B进)>0:
        merge_A出B进 = pd.merge(df_A出.loc[[k],:], df_B进, how = 'outer')
        df_A出B进 = df_A出B进.append(merge_A出B进)
        print(k, end = ' ')
    elif len(df_B进)==0:
        df_A出B空 = df_A出B空.append(df_A出.loc[[k],:])
        print(str(k)+'失败', end = ' ')

36064失败 

In [82]:
def index向后N位的index(df_序列, j_一方, no_N):
    for k in df_序列.index:
        if k >j_一方:
            no_序列号 = len(df_序列.loc[:k] )
            no_目标序列号 = no_序列号 + no_N
            if no_目标序列号 >= len(df_序列):
                index_目标 = df_序列.index.max()
            else:
                index_目标 = df_序列.reset_index().iloc[ no_目标序列号-1 ]['index']
            break
    return index_目标


import datetime

df_匹配 = pd.DataFrame()
df_未匹配 = pd.DataFrame()
df_另一方 = pd.DataFrame(all_流水完整[ (all_流水完整['借贷标志']=='出')  ])
# df_另一方 = pd.DataFrame(all_流水完整[ (all_流水完整['借贷标志']=='出') & ( all_流水完整['对手户名'].map(lambda  x:len(x)>3 or len(x)==0) )  ])
print(str(len(df_一方))+"条，最大索引为"+ str(df_一方.index.max()))
# 差异 days = 1，minutes=10 ,seconds = 10

for j in df_一方.index:
    for N in range(100):
        x = index向后N位的index(df_另一方, j, N)
    
        df_一进一出 = df_另一方[( df_另一方['序号'] <= x ) & ( df_另一方['序号'] > j ) \
            & (df_另一方['查询卡号']== df_一方['查询卡号'].loc[j] ) \

            ]
        if df_一方['交易金额'].loc[j]*(1-0.10) < df_一进一出['交易金额'].sum():
            break
        else:
            continue
    if len(df_一进一出)>0:
        merge_一进一出 = pd.merge(df_一方.loc[[j],:],df_一进一出,how = 'outer')
        df_匹配 = df_匹配.append(merge_一进一出)
        print(str(j)+"-"+str(x), end = ' ')
    elif len(df_一进一出)==0:
        df_未匹配 = df_未匹配.append(df_一方.loc[[j],:])
        print(str(j)+'失败', end = ' ')

with pd.ExcelWriter(output_path + "截止第二天最近一笔出.xlsx") as writer:
    df_匹配.to_excel(writer, sheet_name = '匹配',index = None)
    df_未匹配.to_excel(writer, sheet_name = '未匹配',index = None)

27条，最大索引为37744
36053-36054 36060-36062 36063-36067 36152-36154 36162-36181 36243-36244 36287-36292 36357-36358 36370-36371 36378-36380 36459-36461 36536-36580 36579-36580 36808-36815 36897-36898 36929-36930 37018-37020 37021-37023 37031-37032 37088-37091 37226-37227 37237-37239 37247-37251 37316-37318 37602-37603 37713-37714 37744-37745 

In [1]:
import os
str1 = ''
file_list = []

os.chdir(r"F:\临时测试")
for path_name, dir, files_name in os.walk(os.getcwd()):
    for file in files_name:
        if str1 in file:
            file_list.append(os.path.join(path_name, file))
            
print(len(file_list))
print('\n'.join(file_list))


import pandas as pd
df_group = pd.DataFrame(columns = ['Path','Columns','False','True'])
type1 = '.xls'
type2 = '.csv'
type3 = '.et'
output_path = 'C:\\Users\\ThinkPad\\Desktop\\河南24\\'

for i in range(len(file_list)):
    if type1 in file_list[i]:
        df = pd.read_excel(file_list[i], encoding = "gbk")
        df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    elif type2 in file_list[i]:
        df = pd.read_csv(file_list[i], encoding = 'GB18030')
        df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    else:
        continue
    print(i ,end = ' ')
    df.columns = df.columns.str.strip()
    col_name = df.columns
    bool_S = df.duplicated().value_counts()
    if len(bool_S)==1:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':0}
    elif len(bool_S)==2:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':bool_S[1]}
grouped = df_group.groupby('Columns')
groupname_list = list(grouped.size().index)
print('以下'+str(len(groupname_list))+'种格式数据分类，保留流水、账户类别')
print(grouped.size())

import pandas as pd
import numpy as np
import re
import datetime

all_流水 = pd.DataFrame()
output_path = 'C:\\Users\\ThinkPad\\Desktop\\河南24\\'
if not os.path.exists(output_path + './合并'):
    os.mkdir(output_path + './合并')
for i in range(len(file_list)):
    Efile =  pd.ExcelFile(file_list[i])
    sheet_list = pd.ExcelFile(file_list[i]).sheet_names
    for k in range(0,len(sheet_list)):
        if sheet_list[k] != '':
            df = Efile.parse(sheet_name = sheet_list[k],dtype = 'object')
            df['path'] = file_list[i]
            all_流水 = all_流水.append(df)
            print(str(i)+'-'+str(k),end = ' ')
        else:
            continue
all_流水.to_excel(output_path + '合并\\合并.xlsx',index = None)
print('Done!')

25
F:\临时测试\中国融资租赁有限公司.xlsx
F:\临时测试\中山市宏美迪贸易有限公司.xlsx
F:\临时测试\北京佳禾永利投资管理有限公司.xlsx
F:\临时测试\北京四合雅苑酒店管理有限公司.xlsx
F:\临时测试\北京国益证通投资基金管理有限公司.xlsx
F:\临时测试\北京开元融资租赁有限公司.xlsx
F:\临时测试\北京泰德优胜投资管理有限公司.xlsx
F:\临时测试\北京白金至尊酒业有限公司.xlsx
F:\临时测试\北京长城商务汽车租赁有限公司.xlsx
F:\临时测试\大连百禾资产管理有限公司.xlsx
F:\临时测试\晨新资产管理有限公司.xlsx
F:\临时测试\河南蔡氏餐饮管理有限公司.xlsx
F:\临时测试\河南迈乐加餐厅食品有限公司.xlsx
F:\临时测试\海南国际旅游产业融资租赁股份有限公司.xlsx
F:\临时测试\深圳先锋旅游投资有限公司.xlsx
F:\临时测试\深圳国租海锋保理有限公司.xlsx
F:\临时测试\深圳市天天生鲜农产品有限公司.xlsx
F:\临时测试\深圳市德瑞斯商业保理有限公司.xlsx
F:\临时测试\深圳市西美通信科技有限公司.xlsx
F:\临时测试\深圳盈华商务汽车租赁有限公司.xlsx
F:\临时测试\深圳美福珠宝饰品投资有限公司.xlsx
F:\临时测试\盈华融资租赁有限公司.xlsx
F:\临时测试\益融通商业保理有限公司.xlsx
F:\临时测试\蔡芳新.xlsx
F:\临时测试\黑龙江汇良餐饮投资有限公司.xlsx
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 以下2种格式数据分类，保留流水、账户类别
Columns
(账号, 客户名称, 交易日期, 交易时间, 交易币种, 交易金额, 借贷标志, 余额, 交易机构名称, 客户性质, 交易摘要, 对方账号, 对方户名, 对方行号)           1
(账号, 客户名称, 交易日期, 交易时间, 交易币种, 交易金额, 借贷标志, 余额, 交易机构名称, 客户性质, 交易摘要, 对方账号, 对方户名, 对方行号, 对方行名)    24
dtype: int64
0-0 1-0 2-0 3-0 4-0 5-0 6-0 7-0 8-0 9

In [55]:
import pandas as pd
import numpy as np
import re
import datetime
import os

output_path = r'C:\Users\ThinkPad\Desktop\河南24\合并'
output_path = eval(repr(output_path))+str('\\')

df = pd.read_excel(r"C:\Users\ThinkPad\Desktop\河南\合并\银行交易流水.xlsx",encoding = "gbk",dtype = object)
df = df.applymap(lambda x: re.sub('nan','',str(x)))
df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
df['交易余额'] = df['交易余额'].map(lambda x: float(x))

df['序号'] = df.index
# 24小时标记
df['计时标记'] = ''
all_流水完整 = df

def 计时12to24标记(df_计时):
    for k in range(1,len(df_计时)):
        k1 = k-1
        if df_计时['交易时间'].iloc[k] is None:
            continue
        elif (df_计时['借贷标志'].iloc[k] == '出') and (df_计时['查询卡号'].iloc[k] == df_计时['查询卡号'].iloc[(k1)]) and (df_计时['交易时间'].iloc[k][0:10] == df_计时['交易时间'].iloc[(k1)][0:10]) and (df_计时['交易时间'].iloc[k][11:19] < df_计时['交易时间'].iloc[(k1)][11:19]):
            df_计时['计时标记'].iloc[k] = '+12'
    return df_计时

def 计时24之后标记(df_计时):
    for k in range(1,len(df_计时)):
        k1 = k-1
        if df_计时['交易时间'].iloc[k] is None:
            continue
        elif (df_计时['借贷标志'].iloc[k] == '出') and (df_计时['查询卡号'].iloc[k] == df_计时['查询卡号'].iloc[(k1)]) and (df_计时['交易时间'].iloc[k][0:10] == df_计时['交易时间'].iloc[(k1)][0:10]) and (df_计时['计时标记'].iloc[(k1)]=='+12'):
            df_计时['计时标记'].iloc[k] = '+12'
    return df_计时


def index向后N位的index(df_序列, j_一方, no_N):
    for k in df_序列.index:
        if k >j_一方:
            no_序列号 = len(df_序列.loc[:k] )
            no_目标序列号 = no_序列号 + no_N
            if no_目标序列号 >= len(df_序列):
                index_目标 = df_序列.index.max()
            else:
                index_目标 = df_序列.reset_index().iloc[ no_目标序列号-1 ]['index']
            break
    return index_目标

def 一方匹配另一方(df_一方, all_另一方,sheetname_层数):
    df_一方.index = df_一方['序号'].to_list()
    df_一方 = df_一方[df_一方['借贷标志']=='进'].drop_duplicates(subset = ['序号'],keep = 'last')
    df_匹配 = pd.DataFrame()
    df_未匹配 = pd.DataFrame()
    df_另一方 = pd.DataFrame(all_另一方[ (all_另一方['借贷标志']=='出')  ])
    print(str(len(df_一方))+"条，最大索引为"+ str(df_一方.index.max()))

    for j in df_一方.index:
        for N in range(100):
            x = index向后N位的index(df_另一方, j, N)
    
            df_一进一出 = df_另一方[( df_另一方['序号'] <= x ) & ( df_另一方['序号'] > j ) \
                & (df_另一方['查询卡号']== df_一方['查询卡号'].loc[j] ) \
                ]
            if df_一方['交易金额'].loc[j]*(1-0.10) < df_一进一出['交易金额'].sum(): ############下浮10%###########
                break
            else:
                continue
        if len(df_一进一出)>0:

            
            merge_一进一出 = pd.merge(df_一方.loc[[j],:],df_一进一出,how = 'outer')
            df_匹配 = df_匹配.append(merge_一进一出)
            # 24小时标记
            df_匹配 = 计时12to24标记(df_匹配)
            df_匹配 = 计时24之后标记(df_匹配)
            # 24小时标记
            print(str(j)+"-"+str(x), end = ' ')
        elif len(df_一进一出)==0:
            df_未匹配 = df_未匹配.append(df_一方.loc[[j],:])
            print(str(j)+'失败', end = ' ')

    if not os.path.isfile(output_path +  "追踪转移.xlsx"):
        df_匹配.to_excel(output_path +  "追踪转移.xlsx", sheet_name= sheetname_层数,index = False)
    else:
        with pd.ExcelWriter(output_path +  "追踪转移.xlsx" ,mode='a', engine='openpyxl') as writer:
            df_匹配.to_excel(writer, sheet_name= sheetname_层数,index = False)
    return df_匹配

def AB匹配(df_A,df_B,sheetname_层数):
    df_A.index = df_A['序号'].to_list()
    df_A出B进 = pd.DataFrame()
    df_A出B空 = pd.DataFrame()
    df_A出 = df_A[df_A['借贷标志']=='出'].drop_duplicates(subset = ['序号'],keep = 'last')
    for k in df_A出.index:
        df_B进 = df_B[(df_B['借贷标志']=='进') \
            & ( df_B['交易对方卡号'].isin(df_A出['查询卡号']) ) \
            & (df_B['交易时间']<= str(datetime.datetime.strptime(df_A出['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes = 1) )  ) \
            & (df_B['交易时间']>= str(datetime.datetime.strptime(df_A出['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(seconds = -1) ) ) \
            & (df_B['交易金额'] == df_A出['交易金额'].loc[k])  \
            ]
        if len(df_B进)>0:
            # 24小时标记
            df_B进['计时标记'] = df_A出['计时标记'].loc[k]
            # 24小时标记
            merge_A出B进 = pd.merge(df_A出.loc[[k],:], df_B进, how = 'outer')
            df_A出B进 = df_A出B进.append(merge_A出B进)
            print(k, end = ' ')
        elif len(df_B进)==0:
            df_A出B空 = df_A出B空.append(df_A出.loc[[k],:])
            print(str(k)+'失败', end = ' ')
    if not os.path.isfile(output_path +  "转移未调.xlsx"):
        df_A出B空.to_excel(output_path +  "转移未调.xlsx", sheet_name= sheetname_层数,index = False)
    else:
        with pd.ExcelWriter(output_path +  "转移未调.xlsx" ,mode='a', engine='openpyxl') as writer:
            df_A出B空.to_excel(writer, sheet_name= sheetname_层数,index = False)
    if not os.path.isfile(output_path +  "转移已调.xlsx"):
        df_A出B进.to_excel(output_path +  "转移已调.xlsx", sheet_name= sheetname_层数,index = False)
    else:
        with pd.ExcelWriter(output_path +  "转移已调.xlsx" ,mode='a', engine='openpyxl') as writer:
            df_A出B进.to_excel(writer, sheet_name= sheetname_层数,index = False)
    return df_A出B进

def digui多层(df_层出层进):
    if len(df_层出层进)>0:
        df_某层 = 一方匹配另一方(df_层出层进,all_流水完整,'层')
        df_某层A出B进 = AB匹配(df_某层,all_流水完整,'层')
        digui多层(df_某层A出B进)

In [ ]:
df_原点线索 = all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=10000000 )
                 & (all_流水完整['交易金额']<30000000 )
                 & (all_流水完整['借贷标志']=='进')]
digui多层(df_原点线索)

In [56]:
df_原点线索 = all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=10000000 )
                 & (all_流水完整['交易金额']<30000000 )
                 & (all_流水完整['借贷标志']=='进')]

import os
df_一层 = 一方匹配另一方(df_原点线索,all_流水完整,'一层')
print(len(df_一层))
df_一层A出B进 = AB匹配(df_一层,all_流水完整,'一层')


df_二层 = 一方匹配另一方(df_一层A出B进,all_流水完整,'二层')
print(len(df_二层))
df_二层A出B进 = AB匹配(df_二层,all_流水完整,'二层')

df_三层 = 一方匹配另一方(df_二层A出B进,all_流水完整,'三层')
print(len(df_三层))
df_三层A出B进 = AB匹配(df_三层,all_流水完整,'三层')

df_四层 = 一方匹配另一方(df_三层A出B进,all_流水完整,'四层')
print(len(df_四层))
df_四层A出B进 = AB匹配(df_四层,all_流水完整,'四层')

90条，最大索引为37944
35942-35951 35947-35951 35959-35961 35975-35977 35979-35981 35985-35986 35987-35989 35990-35991 36033-36034 36055-36056 36095-36097 36096-36097 36248-36250 36249-36250 36253-36256 36254-36256 36255-36256 36283-36284 36285-36288 36289-36292 36291-36292 36311-36315 36329-36331 36348-36352 36908-36910 36932-36936 36933-36936 36934-36936 36935-36936 36965-36966 36988-36994 36989-36994 36990-36994 36991-36994 37033-37036 37062-37063 37068-37070 37069-37070 37071-37073 37072-37073 37134-37135 37136-37137 37138-37139 37212-37214 37240-37241 37242-37244 37243-37244 37248-37251 37256-37257 37265-37268 37278-37284 37334-37338 37335-37338 37349-37350 37352-37354 37358-37360 37362-37363 37457-37468 37503-37504 37506-37508 37509-37515 37528-37530 37545-37550 37597-37599 37609-37613 37630-37632 37637-37638 37640-37642 37643-37645 37651-37653 37656-37659 37761-37763 37808-37809 37824-37829 37831-37848 37849-37850 37851-37852 37853-37854 37856-37858 37859-37860 37870-37871 37894-37899 3

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


35961 35976失败 35977失败 35981失败 35986失败 35988 35989失败 35991失败 36034失败 36056 36097失败 36250失败 36256失败 36284失败 36286失败 36288 36292失败 36312 36313失败 36315 36331 36350 36351失败 36352 36910失败 36936 36966失败 36992 36993失败 36994失败 37034失败 37035失败 37036 37063失败 37070 37073 37135 37137 37139 37213失败 37214 37241失败 37244失败 37249失败 37250失败 37251 37257 37266 37267失败 37268失败 37280 37281 37282失败 37283失败 37284失败 37336 37337 37338 37350 37353 37354 37359 37360 37363 37458失败 37460 37462失败 37464 37468失败 37504失败 37507 37508失败 37511 37514 37515失败 37529 37530失败 37546失败 37547失败 37548 37550失败 37598失败 37599 37613失败 37631 37632 37638 37641 37642 37644 37645 37653 37657 37658 37659失败 37762 37763 37809 37825 37826 37827 37828 37829 37832 37833 37834 37835 37836 37837 37838 37839 37840 37841 37842 37843 37844 37845 37846 37847 37848 37850 37852 37854 37857 37858 37860 37871 37895 37896 37897 37898 37899 37910 37911 37912 37913 37914 37915 37916 37917 37918 37919 37920 37921 37922 37923 37945失败 104条，最大索引为41380
3489-3491 

In [59]:
all_流水 = pd.DataFrame()

file_list = ['C:\\Users\\ThinkPad\\Desktop\\河南24\\合并\\追踪转移.xlsx']
for i in range(len(file_list)):
    Efile =  pd.ExcelFile(file_list[i])
    sheet_list = pd.ExcelFile(file_list[i]).sheet_names
    for j in range(0,len(sheet_list)):
        if sheet_list[j] != '':
            df = Efile.parse(sheet_name = sheet_list[j],dtype = 'object')
            df['交易时间'] = df.apply(lambda x: str(datetime.datetime.strptime(x['交易时间'], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours = 12) ) if x['计时标记']=='+12' else x['交易时间'],axis=1)
            if not os.path.isfile(output_path +  "追踪转移24小时计时.xlsx"):
                df.to_excel(output_path +  "追踪转移24小时计时.xlsx", sheet_name= sheet_list[j],index = False)
            else:
                with pd.ExcelWriter(output_path +  "追踪转移24小时计时.xlsx" ,mode='a', engine='openpyxl') as writer:
                    df.to_excel(writer, sheet_name= sheet_list[j],index = False)
            print(str(i)+'-'+str(j),end = ' ')
        else:
            continue


0-0 0-1 0-2 0-3 0-4 0-5 0-6 0-7 

In [31]:
def 计时12to24(df_计时):
    for k in range(1,len(df_计时)):
        k1 = k-1
        if df_计时['交易时间'].iloc[k] is None:
            continue
        elif (df_计时['借贷标志'].iloc[k] == '出') and (df_计时['查询卡号'].iloc[k] == df_计时['查询卡号'].iloc[(k1)]) and (df_计时['交易时间'].iloc[k][0:10] == df_计时['交易时间'].iloc[(k1)][0:10]) and (df_计时['交易时间'].iloc[k][11:19] < df_计时['交易时间'].iloc[(k1)][11:19]):
            df_计时['交易时间'].iloc[k] = str(datetime.datetime.strptime(df_计时['交易时间'].iloc[k], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours = 12) )
    return df_计时

In [61]:
file_list = ['C:\\Users\\ThinkPad\\Desktop\\河南24\\合并\\大于等于一千万小于三千万24小时计时\\追踪转移24小时计时.xlsx']

import pandas as pd
import numpy as np
import re
import datetime

all_流水 = pd.DataFrame()
output_path = 'C:\\Users\\ThinkPad\\Desktop\\河南24\\'
for i in range(len(file_list)):
    Efile =  pd.ExcelFile(file_list[i])
    sheet_list = pd.ExcelFile(file_list[i]).sheet_names
    for k in range(0,len(sheet_list)):
        if sheet_list[k] != '':
            df = Efile.parse(sheet_name = sheet_list[k],dtype = 'object')
            df['path'] = file_list[i]
            all_流水 = all_流水.append(df)
            print(str(i)+'-'+str(k),end = ' ')
        else:
            continue
all_流水.to_excel(output_path + '合并\\追踪转移24小时计时合并.xlsx',index = None)
print('Done!')

0-0 0-1 0-2 0-3 0-4 0-5 0-6 0-7 Done!
